In [ ]:
from openai.types.chat import ChatCompletionSystemMessageParam

system_msg = ChatCompletionSystemMessageParam(
    content="Tu es un assistant spécialisé en cybersécurité."
)

print(system_msg)

In [ ]:
dict(system_msg)
print(system_msg["role"])

# Objectif du notebook :
Valider le fait de pouvoir ajouter plusieurs messages dans un historique de conversation : 
- en ajoutant des conversations à la variable messages qui est une liste soumise au LLM.
- tester d'intégrer ces messages à Langchain pour bénéficier de fonctions déjà développées.

# Historique

In [ ]:
import os
import requests
from openai import OpenAI

base_url = "https://albert.api.etalab.gouv.fr/v1"
api_key = os.getenv("ALBERT_API_KEY")
client = OpenAI(base_url=base_url, api_key=api_key)
session = requests.session()
session.headers = {"Authorization": f"Bearer {api_key}"}

# Historique de conversation sans mécansime supplémentaire

In [ ]:
# Historique des messages de la conversation
messages = []

# Tour 1 : l’utilisateur pose une première question
messages.append(
    {"role": "user", "content": "Quelles sont les principalles attaues informatique ?"}
)
response = client.chat.completions.create(model="albert-large", messages=messages)
# On enregistre la réponse de l'assistant
messages.append({"role": "assistant", "content": response.choices[0].message.content})
print("Réponse 1 :", response.choices[0].message.content)
print("*" * 100)

# Tour 2 : l’utilisateur pose une nouvelle question
messages.append({"role": "user", "content": "Ici qu'est ce qu'un dictionnaire ?"})
response = client.chat.completions.create(model="albert-large", messages=messages)
messages.append({"role": "assistant", "content": response.choices[0].message.content})
print("Réponse 2 :", response.choices[0].message.content)
print("*" * 100)

# Tour 3 : l’utilisateur pose une question qui fait référence à la définition donnée au tour 1
messages.append(
    {"role": "user", "content": "Quelle est la première attaque que tu as cité ?"}
)
response = client.chat.completions.create(model="albert-large", messages=messages)
print("Réponse 3 :", response.choices[0].message.content)
print("*" * 100)

# Intégration Langchain

In [ ]:
!uv add langchain
!uv add langchain_openai

In [ ]:
from typing import List, Dict

try:
    from langchain_openai import ChatOpenAI
    from langchain.memory import ConversationBufferMemory
    from langchain.chains import ConversationChain
except ImportError:
    ChatOpenAI = None  # sera vérifié dans le constructeur


class LangChainHistoryManager:
    """Gère l’historique de conversation en s’appuyant sur LangChain.

    À chaque appel, la question de l’utilisateur est injectée dans la chaîne
    ConversationChain ; la mémoire ConversationBufferMemory stocke tous les messages.
    """

    def __init__(
        self,
        api_key: str,
        base_url: str = "https://albert.api.etalab.gouv.fr/v1",
        model_name: str = "albert-large",
        temperature: float = 0.0,
    ) -> None:
        if ChatOpenAI is None:
            raise ImportError(
                "LangChainHistoryManager nécessite les bibliothèques langchain et langchain-openai. "
                "Installez-les via pip avant utilisation."
            )
        self.memory = ConversationBufferMemory(return_messages=True)
        self.chain = ConversationChain(
            llm=ChatOpenAI(
                model_name=model_name,
                base_url=base_url,
                openai_api_key=api_key,
                temperature=temperature,
                streaming=False,
            ),
            memory=self.memory,
        )

    def ask(self, question: str) -> str:
        """Envoie une question au modèle et met à jour l’historique."""
        response = self.chain.predict(input=question)
        return response

    @property
    def messages(self) -> List[Dict[str, str]]:
        """Expose l’historique sous la forme d’une liste de dicts (role/content)."""
        history = self.memory.load_memory_variables({}).get("history", [])
        result: List[Dict[str, str]] = []
        for msg in history:
            role = getattr(msg, "role", "user")
            result.append({"role": role, "content": msg.content})
        return result

In [ ]:
manager = LangChainHistoryManager(api_key)

rep1 = manager.ask("Quelles sont les principales attaques informatiques ?")
print("Réponse 1 :", rep1)
print("*" * 100)

rep2 = manager.ask("Ici qu'est ce qu'un dictionnaire ?")
print("Réponse 2 :", rep2)
print("*" * 100)

rep3 = manager.ask("Quelle est la première attaque que tu as citée ?")
print("Réponse 3 :", rep3)
print("*" * 100)